In [3]:
import control as ct
import numpy as np
import scipy as sc
import pymor

In [53]:
n = 100;  # VELICINA SUSTAVA
i1 = 0;   # POZICIJA 1. PRIGUSIVACA
i2 = 1;   # POZICIJA 2. PRIGUSIVACA
v1 = 200; # VISKOZNOST 1. PRIGUSIVACA
v2 = 100; # VISKOZNOST 2. PRIGUSIVACA
p = 0.5;  # MJESOVITA H2 MJERA [0, 1]

In [55]:
M  = np.matrix(np.zeros((n, n)));
K  = np.matrix(np.zeros((n, n)));
B  = np.matrix(np.zeros((n, 5)));
Cp = np.matrix(np.zeros((10, n)));
Cv = np.matrix(np.zeros((10, n)));
Zg = 1 / n * np.matrix(np.eye(n));

B[0:5, 0:5]    = np.matrix(np.diag(range(1, 6)));
C1[0:10, 45:55] = np.matrix(np.eye(10));
C2 = C1;

for i in range(n):
    M[i, i] = 200 - 2 * (i + 1) if i < 50 else i + 1 + 50;
    K[i, i] = 200;
    if i > 0:
        K[i, i - 1] = -100;
    
    if i < n - 1:
        K[i, i + 1] = -100;
        
sM = np.sqrt(M);
isM = np.linalg.inv(sM);
E = 0.04 * sM @ np.matrix(sc.linalg.sqrtm(isM @ K @ isM)) * sM;

E[i1, i1] += v1;
E[i2, i2] += v2;

In [56]:
SUSTAV = pymor.models.iosys.SecondOrderModel(M, E, K, B, Cp, Cv)

AttributeError: module 'pymor' has no attribute 'models'